<a href="https://colab.research.google.com/github/gadilshina-ve/BDP_MGPU/blob/main/LW_01/LW_01_Gadilshina_13_09_2024_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests beautifulsoup4 pymongo pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
url = 'https://greenway.icnet.ru/cars-sales-actual-russia.html#null'
response = requests.get(url)
page_content = response.content

In [ ]:
# Проверяем успешность запроса
if response.status_code == 200:
    # Получаем HTML-код страницы
    html_content = response.text

In [ ]:
soup = BeautifulSoup(page_content, 'html.parser')

In [ ]:
# Найти все теги, которые содержат информацию о таблицах или данных
print(soup.prettify())  # Выводим HTML-код страницы для анализа

<!DOCTYPE html>
<html lang="ru">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="–ü–ĺ–Ľ–Ĺ–į—Ź —Ā—ā–į—ā–ł—Ā—ā–ł–ļ–į –Ņ—Ä–ĺ–ī–į–∂ –į–≤—ā–ĺ–ľ–ĺ–Ī–ł–Ľ–Ķ–Ļ –≤ –†–ĺ—Ā—Ā–ł–ł –≤ 2024 –≥–ĺ–ī—É. –Ě–į–≥–Ľ—Ź–ī–Ĺ–ĺ–Ķ –Ņ—Ä–Ķ–ī—Ā—ā–į–≤–Ľ–Ķ–Ĺ–ł–Ķ –ī–į–Ĺ–Ĺ—č—Ö –≤ –≤–ł–ī–Ķ —ā–į–Ī–Ľ–ł—Ü –ł –≥—Ä–į—Ą–ł–ļ–ĺ–≤ –Ņ–ĺ –≥–ĺ–ī–į–ľ –ł –ľ–Ķ—Ā—Ź—Ü–į–ľ." name="description"/>
  <meta content="—Ā—ā–į—ā–ł—Ā—ā–ł–ļ–į –Ņ—Ä–ĺ–ī–į–∂ –į–≤—ā–ĺ–ľ–ĺ–Ī–ł–Ľ–Ķ–Ļ –†–ĺ—Ā—Ā–ł–ł 2024 –≥–ĺ–ī—É —Ü–Ķ–Ĺ—č –Ņ—Ä–ĺ–≥–Ĺ–ĺ–∑—č" name="keywords"/>
  <meta content="website" property="og:type">
   <meta content="https://greenway.icnet.ru/cars-sales-actual-russia.html" property="og:url">
    <meta content="–°—ā–į—ā–ł—Ā—ā–ł–ļ–į –ł —Ä–Ķ–Ļ—ā–ł–Ĺ–≥–ł –Ņ—Ä–ĺ–ī–į–∂ –į–≤—ā–ĺ–ľ–ĺ–Ī–ł–Ľ–Ķ–Ļ –≤ –†–ĺ—Ā—Ā–ł–ł" property="og:title">
     <meta content="–°—ā–į—ā–ł—Ā—ā–ł–ļ–į –Ņ—Ä–ĺ–ī–į–∂ –Ĺ–ĺ–≤—č—Ö –ł –Ī—É –į–≤—ā–ĺ–ľ–ĺ–Ī–ł–Ľ–Ķ–Ļ –≤ –†–ĺ—Ā—Ā–ł–ł.

In [ ]:
table = soup.find('table', {'id': 'unique_id'})  # Ищем таблицу с id='unique_id'
rows = table.find_all('tr')  # Находим все строки таблицы

In [ ]:
data = []
for row in rows[1:]:  # Пропускаем заголовок
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

# Создаем DataFrame
df = pd.DataFrame(data, columns=['Rank', 'Brand', 'Sales_2024', 'Sales_2023', 'Change_percent_2024',
                                 'Brand_2', 'Cumulative_2024', 'Cumulative_2023', 'Cumulative_Change_percent'])

# Заменяем '-' на NaN, чтобы избежать ошибок при преобразовании
df.replace('-', np.nan, inplace=True)

df['Sales_2024'] = df['Sales_2024'].str.replace(',', '.').astype(float)
df['Sales_2023'] = df['Sales_2023'].str.replace(',', '.').astype(float)
df['Change_percent_2024'] = df['Change_percent_2024'].str.replace(',', '.').astype(float)
df['Cumulative_2024'] = df['Cumulative_2024'].str.replace(',', '.').astype(float)
df['Cumulative_2023'] = df['Cumulative_2023'].str.replace(',', '.').astype(float)
df['Cumulative_Change_percent'] = df['Cumulative_Change_percent'].str.replace(',', '.').astype(float)

In [ ]:
print(df)

  Rank    Brand  Sales_2024  Sales_2023  Change_percent_2024  Brand_2  \
0    1     Lada        38.6        28.7                 34.3     Lada   
1    2    Haval        18.6        11.0                 69.1    Haval   
2    3    Chery        16.5        13.4                 23.4    Chery   
3    4    Geely        14.5         8.4                 73.1    Geely   
4    5  Changan        11.4         6.9                 66.4  Changan   
5    6    Omoda         4.7         6.0                -22.1    Omoda   
6    7   Jetour         3.7         1.3                174.8    Exeed   
7    8    Exeed         3.5         4.9                -28.4   Jetour   
8    9   Belgee         3.5         NaN                  NaN   Belgee   
9   10   Jaecoo         2.9         NaN                  NaN   Jaecoo   

   Cumulative_2024  Cumulative_2023  Cumulative_Change_percent  
0            281.6            190.6                       47.7  
1            116.7             58.1                      100.9  
2

In [ ]:
conn = sqlite3.connect('financial_data.db')
df.to_sql('car_sales', conn, if_exists='replace', index=False)

10

In [ ]:
# SQL-запрос для выборки данных для бренда 'Lada'
query = "SELECT * FROM car_sales WHERE Brand = 'Lada'"
df_sqlite1 = pd.read_sql(query1, conn)
print(df_sqlite1)

  Rank    Brand  Sales_2024  Sales_2023  Change_percent_2024  Brand_2  \
0    2    Haval        18.6        11.0                 69.1    Haval   
1    5  Changan        11.4         6.9                 66.4  Changan   

   Cumulative_2024  Cumulative_2023  Cumulative_Change_percent  
0            116.7             58.1                      100.9  
1             71.5             20.9                      242.6  


In [ ]:
#Выборка автомобилей с ростом продаж в 2024 году более чем на 100% по сравнению с 2023 годом:
query2 = "SELECT * FROM car_sales WHERE Cumulative_Change_percent > 100"
df_sqlite2 = pd.read_sql(query2, conn)
print(df_sqlite2)

  Rank    Brand  Sales_2024  Sales_2023  Change_percent_2024  Brand_2  \
0    2    Haval        18.6        11.0                 69.1    Haval   
1    5  Changan        11.4         6.9                 66.4  Changan   

   Cumulative_2024  Cumulative_2023  Cumulative_Change_percent  
0            116.7             58.1                      100.9  
1             71.5             20.9                      242.6  


In [ ]:
#Выборка автомобилей, которые продались лучше в 2023 году, чем в 2024 году:
query3 = "SELECT * FROM car_sales WHERE Sales_2023 > Sales_2024"
df_sqlite3 = pd.read_sql(query3, conn)
print(df_sqlite3)

  Rank  Brand  Sales_2024  Sales_2023  Change_percent_2024 Brand_2  \
0    6  Omoda         4.7         6.0                -22.1   Omoda   
1    8  Exeed         3.5         4.9                -28.4  Jetour   

   Cumulative_2024  Cumulative_2023  Cumulative_Change_percent  
0             35.1             24.3                       44.2  
1             21.7              NaN                        NaN  


In [ ]:
#Выборка топ-5 автомобилей с наибольшим кумулятивным объемом продаж в 2024 году:
query4 = "SELECT * FROM car_sales ORDER BY Cumulative_2024 DESC LIMIT 5"
df_sqlite4 = pd.read_sql(query4, conn)
print(df_sqlite4)

  Rank    Brand  Sales_2024  Sales_2023  Change_percent_2024  Brand_2  \
0    1     Lada        38.6        28.7                 34.3     Lada   
1    2    Haval        18.6        11.0                 69.1    Haval   
2    3    Chery        16.5        13.4                 23.4    Chery   
3    4    Geely        14.5         8.4                 73.1    Geely   
4    5  Changan        11.4         6.9                 66.4  Changan   

   Cumulative_2024  Cumulative_2023  Cumulative_Change_percent  
0            281.6            190.6                       47.7  
1            116.7             58.1                      100.9  
2            102.1             71.3                       43.2  
3             96.7             49.8                       94.0  
4             71.5             20.9                      242.6  


In [ ]:
#Выборка автомобилей с продажами выше среднего в 2024 году:
query5 = "SELECT * FROM car_sales WHERE Sales_2024 > (SELECT AVG(Sales_2024) FROM car_sales)"
df_sqlite5 = pd.read_sql(query5, conn)
print(df_sqlite5)

  Rank  Brand  Sales_2024  Sales_2023  Change_percent_2024 Brand_2  \
0    1   Lada        38.6        28.7                 34.3    Lada   
1    2  Haval        18.6        11.0                 69.1   Haval   
2    3  Chery        16.5        13.4                 23.4   Chery   
3    4  Geely        14.5         8.4                 73.1   Geely   

   Cumulative_2024  Cumulative_2023  Cumulative_Change_percent  
0            281.6            190.6                       47.7  
1            116.7             58.1                      100.9  
2            102.1             71.3                       43.2  
3             96.7             49.8                       94.0  
